In [1]:
import sqlite3
conn = sqlite3.connect("nominations.db")

In [2]:
#look at the schema and the first ten results in nominations
schema = conn.execute("pragma table_info('nominations')").fetchall()
q = "SELECT * FROM nominations LIMIT 10"
first_ten = conn.execute(q).fetchall()

for elem in schema:
    print(elem)
for elem in first_ten:
    print(elem)

(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Category', 'TEXT', 0, None, 0)
(2, 'Nominee', 'TEXT', 0, None, 0)
(3, 'Won', 'INTEGER', 0, None, 0)
(4, 'Movie', 'TEXT', 0, None, 0)
(5, 'Character', 'TEXT', 0, None, 0)
(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(2010, 'Actor -- Supporting 

In [3]:
# create a new table ceremonies with 3 columns: id, year, and host, with id as the primary key
q = "CREATE TABLE ceremonies(id integer, Year integer, Host text, PRIMARY KEY (id))"
conn.execute(q)

OperationalError: table ceremonies already exists

In [4]:
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]


In [5]:
# insert data into ceremonies table
insert_q = "INSERT INTO ceremonies (Year, Host) VALUES (?, ?);"
conn.executemany(insert_q, years_hosts)

In [6]:
#print first 10 results in ceremonies
q = "SELECT * FROM ceremonies LIMIT 10"
conn.execute(q).fetchall()

[(1, 2010, 'Steve Martin'),
 (2, 2009, 'Hugh Jackman'),
 (3, 2008, 'Jon Stewart'),
 (4, 2007, 'Ellen DeGeneres'),
 (5, 2006, 'Jon Stewart'),
 (6, 2005, 'Chris Rock'),
 (7, 2004, 'Billy Crystal'),
 (8, 2003, 'Steve Martin'),
 (9, 2002, 'Whoopi Goldberg'),
 (10, 2001, 'Steve Martin')]

In [7]:
# print schema of ceremonies
conn.execute("pragma table_info(ceremonies)").fetchall()

[(0, 'id', 'integer', 0, None, 1),
 (1, 'Year', 'integer', 0, None, 0),
 (2, 'Host', 'text', 0, None, 0)]

In [8]:
# turn on foreign key constraints to prevent erroneous rows being added
conn.execute("PRAGMA foreign_keys = ON;")

In [9]:
# create a new table nominations_two that will have a ceremony_id column
create_nominations_two = '''create table nominations_two 
(id integer primary key, 
category text, 
nominee text, 
movie text, 
character text, 
won integer,
ceremony_id integer,
foreign key(ceremony_id) references ceremonies(id));
'''
conn.execute(create_nominations_two)

OperationalError: table nominations_two already exists

In [12]:
# select the columns we want from existing nominations table
q = '''
SELECT ceremonies.id as ceremony_id, nominations.category as category, 
nominations.nominee as nominee, nominations.movie as movie, 
nominations.character as character, nominations.won as won
FROM nominations
INNER JOIN ceremonies 
ON nominations.year == ceremonies.year
;
'''
joined_nominations = conn.execute(q).fetchall()

In [13]:
# insert the joined_nominations into nominations_two table
insert_nominations_two = '''INSERT INTO nominations_two
(ceremony_id, category, nominee, movie, character, won)
VALUES (?, ?, ?, ?, ?, ?);'''
conn.executemany(insert_nominations_two, joined_nominations)
print(conn.execute("SELECT * FROM nominations_two LIMIT 5;").fetchall())

[(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1), (2, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 12), (3, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', 0, 1), (4, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', 0, 12), (5, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', 0, 1)]


In [14]:
# delete nominations table and rename nominations_two to nominations
conn.execute("DROP TABLE nominations")
conn.execute("ALTER TABLE nominations_two RENAME TO nominations")

## In the above few steps, we had to create a new table nominations_two with the ceremony_id column and delete the old table nominations because you cannot simply drop columns or change an existing table's schema in SQLite

In [15]:
# Create the tables movies, actors, and movies_actors
create_movies = '''
CREATE TABLE movies (
id INTEGER PRIMARY KEY,
movie TEXT);
'''
conn.execute(create_movies)

create_actors = '''
CREATE TABLE actors (
actor TEXT,
id INTEGER PRIMARY KEY);
'''
conn.execute(create_actors)

create_movies_actors = '''
CREATE TABLE movies_actors (
id INTEGER PRIMARY KEY,
movie_id INTEGER REFERENCES movies(id),
actor_id INTEGER REFERENCES actors(id));
'''
conn.execute(create_movies_actors)

In [ ]:
conn.execute("SELECT * FROM movies_actors LIMIT 5")